In [2]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import optuna
import xgboost as xgb
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, hamming_loss
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier


In [12]:
train=pd.read_csv('/kaggle/input/godatascience2023/Train_one_hot/Train.csv')
VariableDefinitions=pd.read_csv('/kaggle/input/godatascience2023/SampleSubmission.csv')


In [13]:
test=pd.read_csv('/kaggle/input/godatascience2023/Test/Test.csv')


# TO MIN MAX STD MEAN

In [15]:
l=['ip.len','ip.ttl','tcp.offset','tcp.options.timestamp.tsval','tcp.window_size_value','tls.handshake.extensions_ec_point_format.sh',
  'tls.handshake.extensions_length','frame.time_epoch']

## takes alot of time

In [16]:
new_data = []

for i, row in train.iterrows():

    id_val = row['ID']
    new_features = {}
    for feature in l:

        arr = np.array([int(float(x.strip().strip("'"))) if x.strip().strip("'") != '' else 0 for x in row[feature].strip("[]").split(",")])
        min_val = np.min(arr)
        max_val = np.max(arr)
        std_val = np.std(arr)
        mean_val = np.mean(arr)
        sum_val=np.sum(arr)
        
        new_features[f"{feature}_min"] = min_val
        new_features[f"{feature}_max"] = max_val
        new_features[f"{feature}_std"] = std_val
        new_features[f"{feature}_mean"] = mean_val
        new_features[f"{feature}_sum"] = sum_val
    
   
    new_data.append({'ID': id_val, **new_features})


new_df = pd.DataFrame(new_data)

In [19]:
new_data = []

for i, row in test.iterrows():
    id_val = row['ID']
    new_features = {}
    for feature in l:
        arr = np.array([int(float(x.strip().strip("'"))) if x.strip().strip("'") != '' else 0 for x in row[feature].strip("[]").split(",")])
        min_val = np.min(arr)
        max_val = np.max(arr)
        std_val = np.std(arr)
        mean_val = np.mean(arr)
        sum_val=np.sum(arr)
        new_features[f"{feature}_min"] = min_val
        new_features[f"{feature}_max"] = max_val
        new_features[f"{feature}_std"] = std_val
        new_features[f"{feature}_mean"] = mean_val
        new_features[f"{feature}_sum"] = sum_val
    
    new_data.append({'ID': id_val, **new_features})


new_df_test = pd.DataFrame(new_data)

In [ ]:
new_df_test

# Packet Directions

In [20]:
train['packet_directions']

0         ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'I', ...
1         ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'O', 'O', ...
2         ['O', 'I', 'O', 'O', 'I', 'I', 'O', 'I', 'O', ...
3         ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'O', ...
4         ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'I', ...
                                ...                        
324094    ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'O', 'I', ...
324095    ['O', 'I', 'O', 'O', 'I', 'I', 'O', 'I', 'I', ...
324096    ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'O', ...
324097    ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'O', 'I', ...
324098    ['O', 'I', 'O', 'O', 'I', 'I', 'I', 'I', 'I', ...
Name: packet_directions, Length: 324099, dtype: object

## takes alot of time

In [21]:
features = pd.DataFrame(columns=['length', 'num_O', 'num_I', 'num_changes'])
for idx, row in test.iterrows():
    packet_directions = row['packet_directions']
    num_O = packet_directions.count('O')
    num_I = packet_directions.count('I')
    num_changes = sum(1 for i in range(len(packet_directions)-5) if (packet_directions[i] != packet_directions[i+5]) and (packet_directions[i]=='O' or packet_directions[i]=='I'))
    row_features = {'length': len(packet_directions), 'num_O': num_O, 'num_I': num_I, 'num_changes': num_changes}
    features = features.append(row_features, ignore_index=True)
    
print(features.head())

  length num_O num_I num_changes
0    140    15    13          13
1     75     8     7           9
2    185    21    16          19
3    125    13    12          13
4    105     9    12          10


In [22]:
features

,length,num_O,num_I,num_changes
0,140,15,13,13
1,75,8,7,9
2,185,21,16,19
3,125,13,12,13
4,105,9,12,10
...,...,...,...,...
159626,120,14,10,17
159627,140,15,13,11
159628,245,13,36,15
159629,100,9,11,9


In [23]:
test=pd.concat([new_df_test,features],axis=1)

In [24]:
test

,ID,ip.len_min,ip.len_max,ip.len_std,ip.len_mean,ip.len_sum,ip.ttl_min,ip.ttl_max,ip.ttl_std,ip.ttl_mean,...,tls.handshake.extensions_length_sum,frame.time_epoch_min,frame.time_epoch_max,frame.time_epoch_std,frame.time_epoch_mean,frame.time_epoch_sum,length,num_O,num_I,num_changes
0,ID_431730,52,1174,325.141235,226.931034,6581,63,64,0.497317,63.551724,...,449,1675331976,1675331976,0.000000,1.675332e+09,48584627304,140,15,13,13
1,ID_339338,40,1174,481.162005,360.000000,5760,63,64,0.496078,63.562500,...,449,1675695248,1675695249,0.463512,1.675695e+09,26811123979,75,8,7,9
2,ID_085257,40,3418,579.154181,237.473684,9024,63,64,0.493728,63.578947,...,439,1663857583,1663857754,70.311914,1.663858e+09,63226590107,185,21,16,19
3,ID_274334,52,2296,582.399149,384.000000,9984,63,64,0.498519,63.538462,...,449,1675441303,1675441307,1.249852,1.675441e+09,43561473898,125,13,12,13
4,ID_037555,40,1174,425.118314,343.318182,7553,63,128,31.958295,89.590909,...,439,1664378073,1664378075,0.756514,1.664378e+09,36616317631,105,9,12,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159626,ID_111103,52,2908,659.010228,336.600000,8415,63,64,0.489898,63.600000,...,447,1663859481,1663859481,0.000000,1.663859e+09,41596487025,120,14,10,17
159627,ID_260615,52,2296,567.455149,318.586207,9239,63,64,0.497317,63.551724,...,449,1675439170,1675439171,0.304543,1.675439e+09,48587735933,140,15,13,11
159628,ID_017565,40,1174,497.108851,705.900000,35295,63,128,29.184928,81.200000,...,447,1664371926,1664371927,0.140000,1.664372e+09,83218596349,245,13,36,15
159629,ID_352796,40,1174,441.257618,326.000000,6846,63,128,32.463131,93.952381,...,449,1675249292,1675249296,1.743143,1.675249e+09,35180235155,100,9,11,9


In [25]:
features = pd.DataFrame(columns=['length', 'num_O', 'num_I', 'num_changes'])
for idx, row in train.iterrows():
    packet_directions = row['packet_directions']
    num_O = packet_directions.count('O')
    num_I = packet_directions.count('I')
    num_changes = sum(1 for i in range(len(packet_directions)-5) if (packet_directions[i] != packet_directions[i+5]) and (packet_directions[i]=='O' or packet_directions[i]=='I'))
    row_features = {'length': len(packet_directions), 'num_O': num_O, 'num_I': num_I, 'num_changes': num_changes}
    features = features.append(row_features, ignore_index=True)
    
print(features.head())

  length num_O num_I num_changes
0     95     9    10           7
1    220    23    21          32
2    180    20    16          21
3    145    13    16          13
4    145    13    16          13


In [26]:
train=pd.concat([new_df,features],axis=1)

# I saved the training dataset without categorical features in a csv and downloaded it because it takes alot of time to create them from scratch
## Note that I used this dataset to get my final score using the lightgbm implementation below , this dataset is composed with the feature implemented in packet_directions , to min max std mean and in first 3 IP feature ( implementation below) 

In [3]:
trainprev=pd.read_csv('/kaggle/input/withoutcategorical/train (3).csv')
testprev=pd.read_csv('/kaggle/input/withoutcategorical/test (4).csv')

In [4]:
trainprev.head()

,ip.len_min,ip.len_max,ip.len_std,ip.len_mean,ip.len_sum,ip.ttl_min,ip.ttl_max,ip.ttl_std,ip.ttl_mean,ip.ttl_sum,...,frame.time_epoch_max,frame.time_epoch_std,frame.time_epoch_mean,frame.time_epoch_sum,length,num_O,num_I,num_changes,ecn_len,ecn_count
0,40,1174,432.452711,330.050000,6601,63,128,32.500000,95.500000,1910,...,1664377034,0.000000,1.664377e+09,33287540680,95,9,10,7,20,1
1,40,3418,511.481130,183.822222,8272,63,64,0.498888,63.533333,2859,...,1663859300,46.269239,1.663859e+09,74873665442,220,23,21,32,45,0
2,52,3166,557.213383,263.945946,9766,63,64,0.495414,63.567568,2352,...,1663857800,0.000000,1.663858e+09,61562738600,180,20,16,21,37,0
3,40,1174,359.275674,262.700000,7881,63,128,32.427697,93.333333,2800,...,1675352364,1.024153,1.675352e+09,50260570834,145,13,16,13,30,0
4,52,1174,395.489182,308.200000,9246,63,64,0.498888,63.466667,1904,...,1675331425,0.372678,1.675331e+09,50259942745,145,13,16,13,30,14


# First 3 IP features

In [27]:
train1=pd.read_csv('/kaggle/input/godatascience2023/Train_one_hot/Train.csv')

In [28]:
test1=pd.read_csv('/kaggle/input/godatascience2023/Test/Test.csv')

In [29]:
new_data = []
for index, row in train1.iterrows():
    ecn_list = row['ip.dsfield.ecn']
    ecn_len = len([i for i in ecn_list if i in ['2','0']])
    ecn_count = ecn_list.count('2')
    new_data.append([row['ID'], ecn_len, ecn_count])
new_df = pd.DataFrame(new_data, columns=['ID', 'ecn_len', 'ecn_count'])

In [30]:
new_df=new_df.drop(columns=['ID'])

In [31]:
train=pd.concat([train,new_df],axis=1)

In [32]:
new_data = []
for index, row in test1.iterrows():
    ecn_list = row['ip.dsfield.ecn']
    ecn_len = len([i for i in ecn_list if i in ['2','0']])
    ecn_count = ecn_list.count('2')
    new_data.append([row['ID'], ecn_len, ecn_count])


new_df = pd.DataFrame(new_data, columns=['ID', 'ecn_len', 'ecn_count'])

In [33]:
test=pd.concat([test,new_df],axis=1)

In [38]:
train

,ip.len_min,ip.len_max,ip.len_std,ip.len_mean,ip.len_sum,ip.ttl_min,ip.ttl_max,ip.ttl_std,ip.ttl_mean,ip.ttl_sum,...,frame.time_epoch_max,frame.time_epoch_std,frame.time_epoch_mean,frame.time_epoch_sum,length,num_O,num_I,num_changes,ecn_len,ecn_count
0,40,1174,432.452711,330.050000,6601,63,128,32.500000,95.500000,1910,...,1664377034,0.000000,1.664377e+09,33287540680,95,9,10,7,20,1
1,40,3418,511.481130,183.822222,8272,63,64,0.498888,63.533333,2859,...,1663859300,46.269239,1.663859e+09,74873665442,220,23,21,32,45,0
2,52,3166,557.213383,263.945946,9766,63,64,0.495414,63.567568,2352,...,1663857800,0.000000,1.663858e+09,61562738600,180,20,16,21,37,0
3,40,1174,359.275674,262.700000,7881,63,128,32.427697,93.333333,2800,...,1675352364,1.024153,1.675352e+09,50260570834,145,13,16,13,30,0
4,52,1174,395.489182,308.200000,9246,63,64,0.498888,63.466667,1904,...,1675331425,0.372678,1.675331e+09,50259942745,145,13,16,13,30,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324094,52,1174,420.894212,331.058824,5628,63,64,0.499134,63.529412,1080,...,1675333770,1.906100,1.675334e+09,28480674020,80,8,8,9,17,0
324095,52,2296,554.138223,571.500000,28575,63,64,0.496387,63.440000,3172,...,1663859881,0.499600,1.663860e+09,83192994024,245,22,27,21,50,0
324096,52,1174,289.288232,196.206897,5690,63,64,0.497317,63.551724,1843,...,1675333004,0.304543,1.675333e+09,48584657090,140,15,13,17,29,0
324097,52,1174,395.997471,294.717949,11494,63,64,0.499836,63.512821,2477,...,1675686039,2.664694,1.675686e+09,65351755161,190,19,19,15,39,17


In [39]:
trainprev

,ip.len_min,ip.len_max,ip.len_std,ip.len_mean,ip.len_sum,ip.ttl_min,ip.ttl_max,ip.ttl_std,ip.ttl_mean,ip.ttl_sum,...,frame.time_epoch_max,frame.time_epoch_std,frame.time_epoch_mean,frame.time_epoch_sum,length,num_O,num_I,num_changes,ecn_len,ecn_count
0,40,1174,432.452711,330.050000,6601,63,128,32.500000,95.500000,1910,...,1664377034,0.000000,1.664377e+09,33287540680,95,9,10,7,20,1
1,40,3418,511.481130,183.822222,8272,63,64,0.498888,63.533333,2859,...,1663859300,46.269239,1.663859e+09,74873665442,220,23,21,32,45,0
2,52,3166,557.213383,263.945946,9766,63,64,0.495414,63.567568,2352,...,1663857800,0.000000,1.663858e+09,61562738600,180,20,16,21,37,0
3,40,1174,359.275674,262.700000,7881,63,128,32.427697,93.333333,2800,...,1675352364,1.024153,1.675352e+09,50260570834,145,13,16,13,30,0
4,52,1174,395.489182,308.200000,9246,63,64,0.498888,63.466667,1904,...,1675331425,0.372678,1.675331e+09,50259942745,145,13,16,13,30,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324094,52,1174,420.894212,331.058824,5628,63,64,0.499134,63.529412,1080,...,1675333770,1.906100,1.675334e+09,28480674020,80,8,8,9,17,0
324095,52,2296,554.138223,571.500000,28575,63,64,0.496387,63.440000,3172,...,1663859881,0.499600,1.663860e+09,83192994024,245,22,27,21,50,0
324096,52,1174,289.288232,196.206897,5690,63,64,0.497317,63.551724,1843,...,1675333004,0.304543,1.675333e+09,48584657090,140,15,13,17,29,0
324097,52,1174,395.997471,294.717949,11494,63,64,0.499836,63.512821,2477,...,1675686039,2.664694,1.675686e+09,65351755161,190,19,19,15,39,17


# target variable

In [40]:
y=train1[['linux_centos-7',
'linux_opensuse-15.4',
'linux_ubuntu-22.04.1',
'macos_big-sur',
'macos_catalina',
'macos_monterey',
'windows_windows-10',
'windows_windows-11',
'windows_windows-7',
'windows_windows-server-2012',
'windows_windows-server-2012-r2','chrome',
'edge',
'firefox',
'opera']]

In [35]:
y

,linux_centos-7,linux_opensuse-15.4,linux_ubuntu-22.04.1,macos_big-sur,macos_catalina,macos_monterey,windows_windows-10,windows_windows-11,windows_windows-7,windows_windows-server-2012,windows_windows-server-2012-r2,chrome,edge,firefox,opera
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324094,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
324095,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
324096,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
324097,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Models ( note that you can use trainprev from the withoutcategorical file or use the train that is implemented here

In [42]:

#train=train.drop(columns=['ID'])


# second best score ( before final submission score: still gets me 4th place)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
accuracy = hamming_loss(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.011013061812197881


In [44]:
#nocat 0.0111508
#with 88 0.01172
##with 188 feats 0.011873
print(f"Accuracy: {accuracy}")

Accuracy: 0.011013061812197881


# best score ( final submission score)

In [47]:
train['length']=train['length'].astype(int)
train['num_O']=train['num_O'].astype(int)
train['num_I']=train['num_I'].astype(int)
train['num_changes']=train['num_changes'].astype(int)



In [48]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

model = MultiOutputClassifier(lgb.LGBMClassifier(n_estimators=1000, random_state=42))

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = hamming_loss(y_test, y_pred)
print(f"LightGBM Accuracy: {accuracy}")

LightGBM Accuracy: 0.011167335184613802


In [43]:
import matplotlib.pyplot as plt

In [208]:
y.shape

(324099, 15)

# testing decision trees and knn ( lightgbm gives the best score)

In [109]:
dt = tree.DecisionTreeClassifier(random_state = 1)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

accuracy = hamming_loss(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.012292502314100586


In [110]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = hamming_loss(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.03158798724673455


# Making predictions

In [75]:
testprev=testprev.drop(columns=['ID','ID.1'])
preds=model.predict(testprev)

In [78]:
preds=model.predict(testprev)

In [76]:
l=['linux_centos-7',
'linux_opensuse-15.4',
'linux_ubuntu-22.04.1',
'macos_big-sur',
'macos_catalina',
'macos_monterey',
'windows_windows-10',
'windows_windows-11',
'windows_windows-7',
'windows_windows-server-2012',
'windows_windows-server-2012-r2','chrome',
'edge',
'firefox',
'opera']

In [79]:
results = pd.DataFrame(test1['ID'])

for i, col in enumerate(l):
    print(i)
    results[col] = preds[:, i]


results.to_csv('predictions.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [156]:
results

,ID,linux_centos-7,linux_opensuse-15.4,linux_ubuntu-22.04.1,macos_big-sur,macos_catalina,macos_monterey,windows_windows-10,windows_windows-11,windows_windows-7,windows_windows-server-2012,windows_windows-server-2012-r2,chrome,edge,firefox,opera
0,ID_431730,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,ID_339338,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,ID_085257,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,ID_274334,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,ID_037555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159626,ID_111103,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
159627,ID_260615,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
159628,ID_017565,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
159629,ID_352796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [150]:
features.to_csv('feats.csv', index=False)

In [142]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [80]:
from IPython.display import HTML
import pandas as pd
import numpy as np


def create_download_link(title = "Download CSV file", filename = "features.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='predictions.csv')